In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/github_repositories/my_repositories/SteadyStateSchrodingerEquation`


In [2]:
#= Pkg.add("Gridap");
Pkg.add("GridapGmsh");
Pkg.add("Gmsh");
Pkg.add("FileIO");
Pkg.add("LinearAlgebra");
Pkg.add("SparseArrays");
Pkg.add("Arpack"); =#

In [3]:
#= Pkg.status() =#

In [4]:
using Gridap
using GridapGmsh
using Gmsh
using FileIO
using LinearAlgebra
using SparseArrays
using SuiteSparse
using Arpack

In [5]:
include("../src/SteadyStateSchrodingerEquation.jl")

Main.SteadyStateSchrodingerEquation

In [6]:
using .SteadyStateSchrodingerEquation

### Check functions inside `MeshGeneratorFunction.jl`

In [7]:
dom2D=(-25.0,25.0,-25.0,25.0);nxy=(50,50);params_model=((dom2D,nxy));
grid_type="Cartesian2D";

In [8]:
model2D=make_model(grid_type,params_model);

In [9]:
#= using Gridap
writevtk(model2d,"hola") =#

### Check functions inside `BoundaryConditionsFunction.jl`

In [10]:
BC_type="FullDirichlet";
FullDirichlet_values,FullDirichlet_tags=make_boundary_conditions(grid_type,BC_type,ComplexF64);

### Check functions inside `MiscellaneousFunctions.jl`

In [11]:
Ω,dΩ,Γ,dΓ = measures(model2D,3,FullDirichlet_tags)

(BodyFittedTriangulation(), GenericMeasure(), BoundaryTriangulation(), GenericMeasure())

In [12]:
reff = ReferenceFE(lagrangian,Float64,2)

(Lagrangian(), (Float64, 2), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [13]:
VSpace,USpace = FESpaces(model2D,reff,grid_type;BC_type=BC_type,TypeData=ComplexF64)

(UnconstrainedFESpace(), TrialFESpace())

We build functions to write a specific Sturm-Liouville formulation

In [14]:
const ħ=1.0;
const m=1.0;
function eigenvalue_problem_functions(params;switch_potential = "QHO_1D")
    if (switch_potential == "QHO_1D")
        # caso de potencial tipo quantum harmonic oscillator 1D (QHO)
        println("Set quantum harmonic oscillator 1D potential");
        ω,x₁=params;
        p_QHO_1D(x) = 0.5*(ħ*ħ)*(1.0/m);                                      # factor para energía cinética
        q_QHO_1D(x) = 0.5*m*(ω*ω)*(x[1]-x₁)*(x[1]-x₁);                        # oscilador armónico 1D centrado en x₁
        r_QHO_1D(x) = 1.0;
        return p_QHO_1D,q_QHO_1D,r_QHO_1D
    elseif (switch_potential == "QHO_2D")
        # caso de potencial tipo quantum harmonic oscillator 2D (QHO)
        println("Set quantum harmonic oscillator 2D potential");
        ω,x₁,y₁=params;
        p_QHO_2D(x) = 0.5*(ħ*ħ)*(1.0/m);                                       # factor para energía cinética
        q_QHO_2D(x) = 0.5*m*(ω*ω)*((x[1]-x₁)*(x[1]-x₁)+(x[2]-y₁)*(x[2]-y₁));   # oscilador armónico 2D centrado en (x₁,y₁)
        r_QHO_2D(x) = 1.0;
        return p_QHO_2D,q_QHO_2D,r_QHO_2D
    end
end

eigenvalue_problem_functions (generic function with 1 method)

In [15]:
p,q,r = eigenvalue_problem_functions((1.0,0.0,0.0);switch_potential = "QHO_2D")

Set quantum harmonic oscillator 2D potential


(var"#p_QHO_2D#15"(), var"#q_QHO_2D#16"{Float64}(0.0, Core.Box(0.0), Core.Box(1.0)), var"#r_QHO_2D#17"())

### Check functions inside `EigenProblemSolveFunction.jl`

In [16]:
ϵ,ϕ = EigenValuesAndEigenVectors(p,q,r,dΩ,USpace,VSpace;params=(4,10e-9,500,:none,0.0))

(ComplexF64[1.0045798611878358 - 1.7750358144878927e-17im, 2.017535772414723 - 4.9427743429198825e-17im, 2.017535772414729 + 1.0139307186121849e-16im, 3.030491683641623 - 8.535042376344906e-17im], CellField[SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction()])

We build an array with exactly eigenenergie values to quantum 2D harmonic oscillator

In [20]:
function exactly_eigenvalues_2DQHO(num_eigval::Integer)
    ϵ_real_aux=Array{Float64}(undef, num_eigval^2)
    index=1
    for i in 1:num_eigval
        for j in 1:num_eigval
            ϵ_real_aux[index]=((i-1)+(j-1)+1)
            index+=1
        end
    end
    ϵ_real_aux=sort(ϵ_real_aux);
    ϵ_real = ϵ_real_aux[1:num_eigval];
    return ϵ_real
end

exactly_eigenvalues_2DQHO (generic function with 1 method)

In [21]:
ϵ_real=exactly_eigenvalues_2DQHO(length(ϵ))

4-element Vector{Float64}:
 1.0
 2.0
 2.0
 3.0

In [18]:
using Test

In [22]:
@test real(ϵ) ≈ ϵ_real atol=0.1

Test Passed